In [1]:
# piplibs
!pip install --upgrade "ibm-watson>=7.0.0"

In [2]:
# imports
import os
import json
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
from os.path import join, dirname
import subprocess
from ibm_watson import TextToSpeechV1
from ibm_watson.websocket import SynthesizeCallback
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import pygame
pygame.init()



pygame 2.5.2 (SDL 2.28.3, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [3]:
# Speach to text credentials
STT_API_KEY = 'HjmTbvw13GmqFRv7Qo5jfu9Wh2c39AM8TOtE0cyISn9g'
STT_API_URL = 'https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/c793fff1-69a3-4b49-b159-eb479821eccf'
TTS_API_KEY = "b3msZd3C_CGnNru80PUjKlKKHbmLjqT7hEIzZrM7GMyd"
TTS_API_URL = "https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/a871c3e5-ad88-4b4e-941b-04f407861b22"
# BOT_API_KEY = "b8a96e34-da04-44ee-8ded-a91d7962995a"
BOT_API_KEY = "ykdlqdM_0Oiqet-jMgmeIbvQ6Av5q3XKwhSzFv894G_3"
BOT_API_URL = "https://api.us-south.assistant.watson.cloud.ibm.com/instances/574cf614-ba33-4fe7-822b-227573046ad7"

# Set as enviroment variables
os.environ['STT_API_KEY'] = STT_API_KEY
os.environ['STT_API_URL'] = STT_API_URL

os.environ['TTS_API_KEY'] = TTS_API_KEY
os.environ['TTS_APU_URL'] = TTS_API_URL

os.environ['BOT_API_KEY'] = BOT_API_KEY
os.environ['BOT_API_URL'] = BOT_API_URL

### Set up the service for the TTS bot ###

In [4]:
# may need to make an object for v1
authenticator = IAMAuthenticator(TTS_API_KEY)
service = TextToSpeechV1(authenticator=authenticator)
service.set_service_url(TTS_API_URL)

voices = service.list_voices().get_result()
print(json.dumps(voices, indent=2))

{
  "voices": [
    {
      "name": "en-GB_KateVoice",
      "language": "en-GB",
      "gender": "female",
      "description": "Kate: British English female voice.",
      "customizable": true,
      "supported_features": {
        "custom_pronunciation": true,
        "voice_transformation": false
      },
      "url": "https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/a871c3e5-ad88-4b4e-941b-04f407861b22/v1/voices/en-GB_KateVoice"
    },
    {
      "name": "en-US_AllisonVoice",
      "language": "en-US",
      "gender": "female",
      "description": "Allison: American English female voice.",
      "customizable": true,
      "supported_features": {
        "custom_pronunciation": true,
        "voice_transformation": true
      },
      "url": "https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/a871c3e5-ad88-4b4e-941b-04f407861b22/v1/voices/en-US_AllisonVoice"
    },
    {
      "name": "en-US_EmmaExpressive",
      "language": "en-US",
      "gen

In [6]:
def record_user():
    # Sampling frequency
    freq = 44100

    # Recording duration
    duration = 5

    # Start recorder with the given values 
    # of duration and sample frequency
    recording = sd.rec(int(duration * freq), 
                    samplerate=freq, channels=2)

    # Record audio for the given number of seconds
    sd.wait()

    # This will convert the NumPy array to an audio
    # file with the given sampling frequency
    write("recording0.wav", freq, recording)

    # Convert the NumPy array to audio file
    # wv.write("recording1.flac", recording, freq, sampwidth=2)

In [7]:
def decode_audio(file_name):

    ### WINDOWS ###
    subprocess.run(['C:/Users/lincd/Desktop/chatbot_Assign/stt-watson-call.bat'])
    file = open(file_name)
    data = json.load(file)
    return (data["results"][0]['alternatives'][0]['transcript'])

In [42]:
def gen_audio(bot_response_json):

    text_string = 'error'
    output = bot_response_json['output']['generic'][0]['text']
    text_string = output

    print(text_string)
    with open('output.wav', 'wb') as audio_file:
        response = service.synthesize(
            text_string, accept='audio/wav',
            voice="en-US_AllisonVoice").get_result()
        audio_file.write(response.content)

### Set up the chat bot ###

In [27]:
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


assistant_dat = {
    "apikey": "ykdlqdM_0Oiqet-jMgmeIbvQ6Av5q3XKwhSzFv894G_3",
    "url" : "https://api.us-south.assistant.watson.cloud.ibm.com/instances/574cf614-ba33-4fe7-822b-227573046ad7",
    "assistant_id" : "284910d4-d912-4310-9764-183c25702594",

}

authenticator = IAMAuthenticator(assistant_dat['apikey'])
assistant = AssistantV2(
    version='2021-06-14',
    authenticator = authenticator
)

assistant.set_service_url(assistant_dat['url'])

def create_session(): 
    response = assistant.create_session(
        assistant_id= assistant_dat['assistant_id']
    ).get_result()
    assistant_dat['session_id'] = response['session_id']
    print(assistant_dat['session_id'])

def send_response(input_string):
    response = assistant.message(
    assistant_id= assistant_dat['assistant_id'],
    session_id=assistant_dat['session_id'],
    input={
        'message_type': 'text',
        'text': input_string
    }
    ).get_result()

    print(json.dumps(response, indent=2))
    return response

### handel conversation ### 

In [56]:
create_session() # init the convo
def converse():
    record_user()
    user_text = decode_audio(file_name='output-wav.json') # stt
    print(user_text) #test code
    response = send_response(user_text) # talk to bot
    gen_audio(response)
    bot_audio = pygame.mixer.Sound('output.wav')
    bot_audio.play()

8e6ad795-a90d-4701-9f20-d6ae8b4dfc1b


In [60]:
converse()

I have four people in a party 
{
  "output": {
    "intents": [],
    "entities": [
      {
        "entity": "sys-number",
        "location": [
          7,
          11
        ],
        "value": "4",
        "confidence": 1,
        "metadata": {
          "numeric_value": 4
        },
        "interpretation": {
          "numeric_value": 4,
          "subtype": "integer"
        }
      }
    ],
    "generic": [
      {
        "response_type": "text",
        "text": "You are scheduled for dinner on 2024-06-05 at 14:00:00, with a table for 4"
      }
    ]
  },
  "user_id": "8e6ad795-a90d-4701-9f20-d6ae8b4dfc1b"
}
You are scheduled for dinner on 2024-06-05 at 14:00:00, with a table for 4
